In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import sys
sys.path.append('../')

from utils.processing import *
from utils.visualisation import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, cross_validate, StratifiedKFold, GridSearchCV, train_test_split
from sklearn.compose import TransformedTargetRegressor, make_column_transformer
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.decomposition import PCA, TruncatedSVD, SparsePCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.cluster import DBSCAN

np.random.seed(0)

In [3]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 80)

In [4]:
# load data set
df = pd.read_csv('../data/train.csv')
df['subzone'] = df['subzone'].astype('category', copy=False)
df['planning_area'] = df['planning_area'].astype('category', copy=False)


df_train_orig, df_validate_orig = train_test_split(df, test_size=0.30)
df_train_orig.iloc[0]
df_train_orig = df.copy()

In [5]:
# Utility functions

def prepare_data_for_regression(df, drop_columns=[]):
    adfs = read_aux_csv('../data')
    df = join_aux(df, adfs)
    #df['region_c'] = np.where(df['region'] == 'c', 1, 0)
    df = df.drop(columns=drop_columns)

    return df

def rmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    score = np.sqrt(mse)
    return score

def rmse_scorer():
    return make_scorer(rmse, greater_is_better=False)

# Constants
initial_drop_columns = [
    'address',
    'title',
    'listing_id',
    'property_name',
    'available_unit_types',
    'property_details_url',
    'elevation',
    'tenure',
    'property_type',
    'floor_level',
    'furnishing',
]

drop_columns = initial_drop_columns + [
    #'subzone',
    #'planning_area',
    #'region',
    'lat',
    'lng',
    'floor_level_ground',
    'floor_level_high',
    'floor_level_low',
    'floor_level_mid',
    'floor_level_penthouse',
    'floor_level_top',
    'furnishing_partial',
    'furnishing_unfurnished',
    'furnishing_unspecified',
    'line_cc',
    'line_ce',
    'line_cg',
    'line_dt',
    'line_ew',
    'line_ne',
    'line_ns',
    'line_te',
    'gep_pri_sch_within_1km',
    'gep_pri_sch_within_1km_2km',
    'cc_type_CR',
    'cc_type_IEBP',
    'cc_type_IHL',
]

In [6]:
# Prepare training data set
df_train = preprocess(df_train_orig.copy())
df_train = prepare_data_for_regression(df_train, drop_columns=drop_columns)
df_train

,built_year,num_beds,num_baths,size_sqft,total_num_units,subzone,planning_area,price,property_type_private,property_type_public,tenure_high_year,tenure_low_year,nearest_mrt_distance_in_km,nearest_pri_sch_distance_in_km,nearest_gep_pri_sch_distance_in_km,gep_pri_sch_outside_2km,pri_sch_within_500m,pri_sch_outside_500m,nearest_com_centre_distance_in_km,cc_type_BN,nearest_mall_distance_in_km,area_size,population,density,region
0,1988.0,3.0,2.0,1115.0,116.0,yishun south,yishun,514500.0,0,1,0,0,0.574204,0.275582,6.194099,1,1,0,3.338999,0,0.621334,1.3402,42240,31517.683928,n
1,1992.0,4.0,2.0,1575.0,NaN,serangoon north,serangoon,995400.0,0,1,0,1,1.734303,0.122925,0.122925,0,1,0,2.401882,1,0.552544,0.6847,15940,23280.268731,ne
2,2022.0,4.0,6.0,3070.0,56.0,mountbatten,marine parade,8485000.0,1,0,1,0,1.319766,0.891475,1.872702,0,0,1,2.171328,1,0.824208,1.7119,9980,5829.779777,c
3,2023.0,3.0,2.0,958.0,638.0,farrer court,bukit timah,2626000.0,1,0,1,0,0.726006,1.090371,1.090371,0,0,1,1.605928,1,0.907063,0.5588,6180,11059.413028,c
4,2026.0,2.0,1.0,732.0,351.0,anson,downtown core,1764000.0,1,0,0,1,0.371115,0.464835,2.742534,1,1,0,1.869574,0,0.430415,0.1032,80,775.193798,c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20140,2026.0,2.0,2.0,635.0,605.0,tagore,ang mo kio,1050000.0,1,0,0,1,0.149941,0.790370,4.691173,1,0,1,3.163579,0,2.014133,3.3342,7950,2384.380061,ne
20141,2026.0,2.0,2.0,883.0,137.0,lavender,kallang,2087400.0,1,0,1,0,0.443603,0.660929,2.178475,1,0,1,1.641446,1,0.535185,0.7569,9690,12802.219580,c
20142,2023.0,4.0,4.0,1378.0,340.0,moulmein,novena,4193700.0,1,0,1,0,0.424244,0.367552,0.367552,0,1,0,0.856170,1,0.764410,1.3475,9300,6901.669759,c
20143,2017.0,3.0,2.0,1205.0,402.0,woodlands east,woodlands,754800.0,0,1,0,1,0.631718,0.230075,10.658711,1,1,0,2.165768,0,0.601720,2.5535,98980,38762.482867,n


In [7]:
X_train = df_train.drop('price', axis=1)
y_train = df_train['price']

In [8]:
ct = make_column_transformer(
                    (make_pipeline(KNNImputer(), StandardScaler()), ['num_beds', 'num_baths', 'built_year', 'total_num_units']),
                    (StandardScaler(), ['nearest_mrt_distance_in_km', 'nearest_pri_sch_distance_in_km', 'nearest_gep_pri_sch_distance_in_km', 'nearest_com_centre_distance_in_km', 'nearest_mall_distance_in_km', 'area_size', 'population', 'density']),
                    (OneHotEncoder(sparse=False, handle_unknown='ignore'), ["region", "subzone", "planning_area"]),
                    remainder='passthrough')

def ttr(pipeline):
    return TransformedTargetRegressor(regressor=pipeline, transformer=StandardScaler())

def transform_data(df):
    p = make_pipeline(ct)
    p.fit(df)
    
    return pd.DataFrame(data=p.transform(df), columns=p.get_feature_names_out())

In [9]:
t_X_train = transform_data(X_train)
t_X_train.head()

,pipeline__num_beds,pipeline__num_baths,pipeline__built_year,pipeline__total_num_units,standardscaler__nearest_mrt_distance_in_km,standardscaler__nearest_pri_sch_distance_in_km,standardscaler__nearest_gep_pri_sch_distance_in_km,standardscaler__nearest_com_centre_distance_in_km,standardscaler__nearest_mall_distance_in_km,standardscaler__area_size,standardscaler__population,standardscaler__density,onehotencoder__region_c,onehotencoder__region_e,onehotencoder__region_n,onehotencoder__region_ne,onehotencoder__region_w,onehotencoder__subzone_admiralty,onehotencoder__subzone_alexandra hill,onehotencoder__subzone_alexandra north,onehotencoder__subzone_aljunied,onehotencoder__subzone_anak bukit,onehotencoder__subzone_anchorvale,onehotencoder__subzone_ang mo kio town centre,onehotencoder__subzone_anson,...,onehotencoder__planning_area_punggol,onehotencoder__planning_area_queenstown,onehotencoder__planning_area_river valley,onehotencoder__planning_area_rochor,onehotencoder__planning_area_seletar,onehotencoder__planning_area_sembawang,onehotencoder__planning_area_sengkang,onehotencoder__planning_area_serangoon,onehotencoder__planning_area_singapore river,onehotencoder__planning_area_southern islands,onehotencoder__planning_area_tampines,onehotencoder__planning_area_tanglin,onehotencoder__planning_area_tengah,onehotencoder__planning_area_toa payoh,onehotencoder__planning_area_woodlands,onehotencoder__planning_area_yishun,remainder__size_sqft,remainder__property_type_private,remainder__property_type_public,remainder__tenure_high_year,remainder__tenure_low_year,remainder__gep_pri_sch_outside_2km,remainder__pri_sch_within_500m,remainder__pri_sch_outside_500m,remainder__cc_type_BN
0,-0.092900,-0.439385,-1.440578,-0.750520,-0.387375,-0.870002,0.992342,1.332502,-0.322523,-0.162035,0.851992,1.392520,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1115.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
1,0.686944,-0.439385,-1.185038,-0.901738,1.698473,-1.154203,-1.187419,0.454867,-0.451613,-0.410230,-0.264521,0.704546,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1575.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
2,0.686944,2.279902,0.731518,-0.934932,0.953139,0.276600,-0.559189,0.238947,0.058179,-0.021297,-0.517540,-0.752887,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3070.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
3,-0.092900,-0.439385,0.795403,0.853862,-0.114437,0.646883,-0.840072,-0.290565,0.213661,-0.457900,-0.678862,-0.316117,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,958.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
4,-0.872744,-1.119207,0.987059,-0.028241,-0.752526,-0.517673,-0.246889,-0.043654,-0.680793,-0.630405,-0.937824,-1.175036,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,732.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0


In [10]:
t_X_train[t_X_train.isna().any(axis=1)]

,pipeline__num_beds,pipeline__num_baths,pipeline__built_year,pipeline__total_num_units,standardscaler__nearest_mrt_distance_in_km,standardscaler__nearest_pri_sch_distance_in_km,standardscaler__nearest_gep_pri_sch_distance_in_km,standardscaler__nearest_com_centre_distance_in_km,standardscaler__nearest_mall_distance_in_km,standardscaler__area_size,standardscaler__population,standardscaler__density,onehotencoder__region_c,onehotencoder__region_e,onehotencoder__region_n,onehotencoder__region_ne,onehotencoder__region_w,onehotencoder__subzone_admiralty,onehotencoder__subzone_alexandra hill,onehotencoder__subzone_alexandra north,onehotencoder__subzone_aljunied,onehotencoder__subzone_anak bukit,onehotencoder__subzone_anchorvale,onehotencoder__subzone_ang mo kio town centre,onehotencoder__subzone_anson,...,onehotencoder__planning_area_punggol,onehotencoder__planning_area_queenstown,onehotencoder__planning_area_river valley,onehotencoder__planning_area_rochor,onehotencoder__planning_area_seletar,onehotencoder__planning_area_sembawang,onehotencoder__planning_area_sengkang,onehotencoder__planning_area_serangoon,onehotencoder__planning_area_singapore river,onehotencoder__planning_area_southern islands,onehotencoder__planning_area_tampines,onehotencoder__planning_area_tanglin,onehotencoder__planning_area_tengah,onehotencoder__planning_area_toa payoh,onehotencoder__planning_area_woodlands,onehotencoder__planning_area_yishun,remainder__size_sqft,remainder__property_type_private,remainder__property_type_public,remainder__tenure_high_year,remainder__tenure_low_year,remainder__gep_pri_sch_outside_2km,remainder__pri_sch_within_500m,remainder__pri_sch_outside_500m,remainder__cc_type_BN


In [11]:
def run_models(models, X, y):    
    model_scores = dict()
    
    for i, model in enumerate(models):
        regr = ttr(make_pipeline(ct, model))
  
        scores = cross_validate(regr, X, y, cv=5, scoring=rmse_scorer())
        regr.fit(X, y)
        model_scores[model.__class__.__name__ + str(i)] = [regr, scores]
        print(model.__class__.__name__, scores['test_score'].mean())
        
    return model_scores

In [12]:
models = [
            LinearRegression(),
            Lasso(),
            Ridge(),
            KNeighborsRegressor(),
            xgb.XGBRegressor(),
        ]


model_scores = run_models(models, X_train, y_train)

LinearRegression -2611207422498.6265
Lasso -3319737.218619094
Ridge -2831219.27468961
KNeighborsRegressor -2695453.476101221
XGBRegressor -1847680.909688056


In [13]:
y_validate = df_validate_orig['price']
X_validate = prepare_data_for_regression(preprocess(df_validate_orig.drop('price', axis=1), is_target=True), drop_columns=drop_columns)

t_X_v = transform_data(X_validate)
t_X_v.head()

,pipeline__num_beds,pipeline__num_baths,pipeline__built_year,pipeline__total_num_units,standardscaler__nearest_mrt_distance_in_km,standardscaler__nearest_pri_sch_distance_in_km,standardscaler__nearest_gep_pri_sch_distance_in_km,standardscaler__nearest_com_centre_distance_in_km,standardscaler__nearest_mall_distance_in_km,standardscaler__area_size,standardscaler__population,standardscaler__density,onehotencoder__region_c,onehotencoder__region_e,onehotencoder__region_n,onehotencoder__region_ne,onehotencoder__region_w,onehotencoder__subzone_admiralty,onehotencoder__subzone_alexandra hill,onehotencoder__subzone_alexandra north,onehotencoder__subzone_aljunied,onehotencoder__subzone_anak bukit,onehotencoder__subzone_anchorvale,onehotencoder__subzone_ang mo kio town centre,onehotencoder__subzone_anson,...,onehotencoder__planning_area_punggol,onehotencoder__planning_area_queenstown,onehotencoder__planning_area_river valley,onehotencoder__planning_area_rochor,onehotencoder__planning_area_seletar,onehotencoder__planning_area_sembawang,onehotencoder__planning_area_sengkang,onehotencoder__planning_area_serangoon,onehotencoder__planning_area_singapore river,onehotencoder__planning_area_southern islands,onehotencoder__planning_area_tampines,onehotencoder__planning_area_tanglin,onehotencoder__planning_area_tengah,onehotencoder__planning_area_toa payoh,onehotencoder__planning_area_woodlands,onehotencoder__planning_area_yishun,remainder__size_sqft,remainder__property_type_private,remainder__property_type_public,remainder__tenure_high_year,remainder__tenure_low_year,remainder__gep_pri_sch_outside_2km,remainder__pri_sch_within_500m,remainder__pri_sch_outside_500m,remainder__cc_type_BN
0,0.682822,0.916767,0.790116,-1.000407,-0.485358,0.438822,-0.553770,-0.650078,-0.555592,0.058527,0.106832,-0.174706,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2153.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
1,-0.862389,-1.106424,0.917894,-0.143403,-0.081178,0.909779,-0.751929,-0.400462,0.083827,0.091878,-0.667007,-0.972863,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,614.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
2,-0.089783,-0.432027,0.406781,0.160307,-0.088027,-0.923569,2.873543,-0.428278,-0.240812,0.282934,3.231257,1.995580,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,991.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0
3,2.228032,0.916767,-2.148782,-0.997400,0.091244,-0.679120,-0.776326,-0.975115,-1.060358,0.050365,0.427826,0.167178,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3240.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
4,-0.089783,-0.432027,0.087336,1.187509,0.828457,-0.810085,1.150104,0.963756,-0.032994,-0.007991,1.617441,1.622717,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,990.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0


In [14]:
t_X_v[t_X_v.isna().any(axis=1)]

,pipeline__num_beds,pipeline__num_baths,pipeline__built_year,pipeline__total_num_units,standardscaler__nearest_mrt_distance_in_km,standardscaler__nearest_pri_sch_distance_in_km,standardscaler__nearest_gep_pri_sch_distance_in_km,standardscaler__nearest_com_centre_distance_in_km,standardscaler__nearest_mall_distance_in_km,standardscaler__area_size,standardscaler__population,standardscaler__density,onehotencoder__region_c,onehotencoder__region_e,onehotencoder__region_n,onehotencoder__region_ne,onehotencoder__region_w,onehotencoder__subzone_admiralty,onehotencoder__subzone_alexandra hill,onehotencoder__subzone_alexandra north,onehotencoder__subzone_aljunied,onehotencoder__subzone_anak bukit,onehotencoder__subzone_anchorvale,onehotencoder__subzone_ang mo kio town centre,onehotencoder__subzone_anson,...,onehotencoder__planning_area_punggol,onehotencoder__planning_area_queenstown,onehotencoder__planning_area_river valley,onehotencoder__planning_area_rochor,onehotencoder__planning_area_seletar,onehotencoder__planning_area_sembawang,onehotencoder__planning_area_sengkang,onehotencoder__planning_area_serangoon,onehotencoder__planning_area_singapore river,onehotencoder__planning_area_southern islands,onehotencoder__planning_area_tampines,onehotencoder__planning_area_tanglin,onehotencoder__planning_area_tengah,onehotencoder__planning_area_toa payoh,onehotencoder__planning_area_woodlands,onehotencoder__planning_area_yishun,remainder__size_sqft,remainder__property_type_private,remainder__property_type_public,remainder__tenure_high_year,remainder__tenure_low_year,remainder__gep_pri_sch_outside_2km,remainder__pri_sch_within_500m,remainder__pri_sch_outside_500m,remainder__cc_type_BN


In [15]:
y_validate = df_validate_orig['price']
X_validate = prepare_data_for_regression(preprocess(df_validate_orig.drop('price', axis=1), is_target=True))

for k, v in model_scores.items():
    regr, scores = v
    predict_validate = regr.predict(X_validate)
    
    print(k, rmse(y_validate, predict_validate))
    
# with lat lng
# LinearRegression0 22801849.758433893
# Lasso1 22801820.600159608
# Ridge2 22801398.1202299
# KNeighborsRegressor3 2279910.9277904155
# XGBRegressor4 1848489.1003341698

LinearRegression0 21633544.86123581
Lasso1 31356180.43030959
Ridge2 21658947.00557342
KNeighborsRegressor3 2593092.4985592877
XGBRegressor4 1449602.8223770384


In [16]:
# Retrain using best model
fmodel = make_pipeline(ct, LinearDiscriminantAnalysis(), xgb.XGBRegressor())

df_ftrain = pd.read_csv('../data/train.csv')
df_ftrain = preprocess(df_ftrain)
df_ftrain = prepare_data_for_regression(df_ftrain, drop_columns=drop_columns)
df_ftrain.head()

X_ftrain = df_ftrain.drop('price', axis=1)
y_ftrain = df_ftrain['price']

fmodel.fit(X_ftrain, y_ftrain)
predict_ftrain = fmodel.predict(X_ftrain)
print(rmse(y_ftrain, predict_ftrain))

426546.04703685484


In [17]:
df_test = preprocess(pd.read_csv('../data/test.csv'), is_target=True)

# further preprocessing
X_test = prepare_data_for_regression(df_test, drop_columns=drop_columns)

y_predict = fmodel.predict(X_test)

X_test['Predicted'] = y_predict

In [18]:
y_predict

array([1191702.5, 1471153.9, 1318635.9, ..., 3428703. ,  828512.9,
       4269205.5], dtype=float32)

In [19]:
submission = X_test[['Predicted']]
submission.to_csv('submission3.csv', index=True, index_label='id', header=True, columns=['Predicted'])